In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
x_train = mnist.train.images
y_train = mnist.train.labels
x_test = mnist.test.images
y_test = mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
#for NN layers
class layer:
    def __init__(self, inputs, in_size, out_size, activation_function=None):
#         self.W = tf.Variable(tf.zeros([in_size, out_size]))
        self.W = tf.Variable(tf.random_normal([in_size, out_size]))
#         self.b = tf.Variable(tf.zeros([1,out_size]))
        self.b = tf.Variable(tf.constant(0.1, shape=[1,out_size]))
        self.Wx_plus_b = tf.matmul(inputs, self.W) + self.b
        self.activation_function = activation_function
    def output(self):
        if self.activation_function == None:
            result = self.Wx_plus_b
        else :
            result = self.activation_function(self.Wx_plus_b)
        return result

In [4]:
# for convolution
class convolution:
    def __init__(self, shape):
        self.W = tf.Variable(tf.truncated_normal(shape=shape, stddev=0.1))
        self.b = tf.Variable(tf.constant(0.1, shape=[shape[3]]))
    def conv2d(self, inputs, padding='SAME'):
        return tf.nn.conv2d(inputs, self.W, strides=[1,1,1,1], padding=padding)
    def max_pooling_nxn(self, inputs, n):
        return tf.nn.max_pool(inputs, ksize=[1,n,n,1], strides=[1,n,n,1], padding='SAME')
    def conv_and_pooling(self, inputs, activation_function=None, pooling_size=2):
        if activation_function==None:
            h_conv =self.conv2d(inputs)+self.b
        else:
            h_conv = activation_function(self.conv2d(inputs)+self.b)    
        return self.max_pooling_nxn(h_conv, pooling_size)

In [5]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

#for dropout
keep_prob = tf.placeholder(tf.float32)

#dealing eith inputs. 
# -1 for not considering input number of images
# 28,28 size per iamge
# 1 for channel. 1 for grayscale ; 3 for color image
x_image=tf.reshape(x,[-1,28,28,1])

In [6]:
#AlexNet => 5 convolution layers & 3 fully connected neural network

conv1 = convolution([3,3,1,24])
conv2 = convolution([3,3,24,48])
conv3 = convolution([3,3,48,144])
conv4 = convolution([2,2,144,288])
conv5 = convolution([2,2,288,576])

output_conv1 = conv1.conv_and_pooling(x_image, tf.nn.relu)
output_conv2 = conv2.conv_and_pooling(output_conv1, tf.nn.relu)
output_conv3 = conv3.conv_and_pooling(output_conv2, tf.nn.relu, pooling_size=2)
output_conv4 = conv4.conv_and_pooling(output_conv3, tf.nn.relu, pooling_size=2)
output_conv5 = conv5.conv_and_pooling(output_conv4, tf.nn.relu, pooling_size=2)

# h_pool_flat = tf.reshape(output_conv2, [-1,7*7*40])
h_pool_flat = tf.reshape(output_conv5, [-1,1*1*576])

# layer1 = layer(h_pool_flat, 7*7*64, 500, tf.nn.relu)
# layer1 = layer(h_pool_flat, 7*7*64, 10, tf.nn.softmax)
# layer1 = layer(h_pool_flat, 7*7*40, 10, tf.nn.softmax)
layer1 = layer(h_pool_flat, 1*1*576, 100, tf.nn.sigmoid)
# layer2 = layer(layer1.output(), 400, 100, tf.nn.sigmoid)
h_drop = tf.nn.dropout(layer1.output(), keep_prob)
layer3 = layer(h_drop, 100, 10, tf.nn.softmax)
# layer3 = layer(layer1.output(), 100, 10, tf.nn.softmax)
# layer2 = layer(layer1.output(), 500, 100, tf.nn.relu)
# layer3 = layer(layer2.output(), 100, 10, tf.nn.softmax)

# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=layer1.output()))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=layer3.output()))
# train_step = tf.train.GradientDescentOptimizer(0.3).minimize(loss)
train_step =  tf.train.MomentumOptimizer(0.003 , 0.7).minimize(loss)
# train_step = tf.train.AdamOptimizer(0.003).minimize(loss)


correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(layer3.output(), 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
init = tf.global_variables_initializer()

# sess = tf.Session()
gpu_options = tf.GPUOptions(allow_growth=True)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
sess.run(init)

batch_size = 50
batches = x_train.shape[0]//batch_size


In [8]:
# for epoch in range(31):
#     print "start "+str(epoch)
#     for batch in range(batches):
#         sess.run(train_step, feed_dict={x: x_train[batch_size*batch:batch_size*(batch+1)], y: y_train[batch_size*batch:batch_size*(batch+1)]})
#     if epoch%2==0:
#         print "epoch: "+str(epoch)+" loss: "+str(sess.run(loss, feed_dict={x: x_train, y: y_train}))+" accuracy: "+str(sess.run(accuracy, feed_dict={x: x_train, y: y_train}))
# print "Test accuracy: "+str(sess.run(accuracy, feed_dict={x: x_test, y: y_test}))


for epoch in range(31):
    print "start "+str(epoch)
    for batch in range(batches):
        sess.run(train_step, feed_dict={x: x_train[batch_size*batch:batch_size*(batch+1)], y: y_train[batch_size*batch:batch_size*(batch+1)], keep_prob: 0.6})
    if epoch%2==0:
        print "epoch: "+str(epoch)+" loss: "+str(sess.run(loss, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))+" accuracy: "+str(sess.run(accuracy, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))
print "Test accuracy: "+str(sess.run(accuracy, feed_dict={x: x_test, y: y_test, keep_prob: 1.0}))

start 0
epoch: 0 loss: 2.24625 accuracy: 0.204255
start 1
start 2
epoch: 2 loss: 1.83276 accuracy: 0.635436
start 3
start 4
epoch: 4 loss: 1.68118 accuracy: 0.781036
start 5
start 6
epoch: 6 loss: 1.60342 accuracy: 0.8578
start 7
start 8
epoch: 8 loss: 1.59069 accuracy: 0.869982
start 9
start 10
epoch: 10 loss: 1.5898 accuracy: 0.871818
start 11
start 12
epoch: 12 loss: 1.58061 accuracy: 0.880327
start 13
start 14
epoch: 14 loss: 1.55397 accuracy: 0.907764
start 15
start 16
epoch: 16 loss: 1.49643 accuracy: 0.965436
start 17
start 18
epoch: 18 loss: 1.4889 accuracy: 0.973055
start 19
start 20
epoch: 20 loss: 1.48712 accuracy: 0.974582
start 21
start 22
epoch: 22 loss: 1.48561 accuracy: 0.976091
start 23
start 24
epoch: 24 loss: 1.48423 accuracy: 0.977491
start 25
start 26
epoch: 26 loss: 1.48067 accuracy: 0.980891
start 27
start 28
epoch: 28 loss: 1.47944 accuracy: 0.982218
start 29
start 30
epoch: 30 loss: 1.47811 accuracy: 0.983455
Test accuracy: 0.983


In [9]:
# for epoch in range(21):
#     print "start "+str(epoch)
#     for batch in range(batches):
#         sess.run(train_step, feed_dict={x: x_train[batch_size*batch:batch_size*(batch+1)], y: y_train[batch_size*batch:batch_size*(batch+1)]})
#     if epoch%2==0:
#         print "epoch: "+str(epoch)+" loss: "+str(sess.run(loss, feed_dict={x: x_train, y: y_train}))+" accuracy: "+str(sess.run(accuracy, feed_dict={x: x_train, y: y_train}))
# print "Test accuracy: "+str(sess.run(accuracy, feed_dict={x: x_test, y: y_test}))


for epoch in range(31):
    print "start "+str(epoch)
    for batch in range(batches):
        sess.run(train_step, feed_dict={x: x_train[batch_size*batch:batch_size*(batch+1)], y: y_train[batch_size*batch:batch_size*(batch+1)], keep_prob: 0.6})
    if epoch%2==0:
        print "epoch: "+str(epoch)+" loss: "+str(sess.run(loss, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))+" accuracy: "+str(sess.run(accuracy, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))
print "Test accuracy: "+str(sess.run(accuracy, feed_dict={x: x_test, y: y_test, keep_prob: 1.0}))

start 0
epoch: 0 loss: 1.47877 accuracy: 0.982709
start 1
start 2
epoch: 2 loss: 1.47594 accuracy: 0.985364
start 3
start 4
epoch: 4 loss: 1.47531 accuracy: 0.985964
start 5
start 6
epoch: 6 loss: 1.4751 accuracy: 0.986164
start 7
start 8
epoch: 8 loss: 1.47416 accuracy: 0.987291
start 9
start 10
epoch: 10 loss: 1.47409 accuracy: 0.987473
start 11
start 12
epoch: 12 loss: 1.47358 accuracy: 0.987691
start 13
start 14
epoch: 14 loss: 1.47287 accuracy: 0.988545
start 15
start 16
epoch: 16 loss: 1.4717 accuracy: 0.989618
start 17
start 18
epoch: 18 loss: 1.47207 accuracy: 0.989127
start 19
start 20
epoch: 20 loss: 1.4714 accuracy: 0.989909
start 21
start 22
epoch: 22 loss: 1.47087 accuracy: 0.990636
start 23
start 24
epoch: 24 loss: 1.47055 accuracy: 0.990836
start 25
start 26
epoch: 26 loss: 1.47025 accuracy: 0.991073
start 27
start 28
epoch: 28 loss: 1.46969 accuracy: 0.9916
start 29
start 30
epoch: 30 loss: 1.46944 accuracy: 0.991891
Test accuracy: 0.9876


In [10]:
for epoch in range(31):
    print "start "+str(epoch)
    for batch in range(batches):
        sess.run(train_step, feed_dict={x: x_train[batch_size*batch:batch_size*(batch+1)], y: y_train[batch_size*batch:batch_size*(batch+1)], keep_prob: 0.6})
    if epoch%2==0:
        print "epoch: "+str(epoch)+" loss: "+str(sess.run(loss, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))+" accuracy: "+str(sess.run(accuracy, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))
print "Test accuracy: "+str(sess.run(accuracy, feed_dict={x: x_test, y: y_test, keep_prob: 1.0}))

start 0
epoch: 0 loss: 1.46909 accuracy: 0.992236
start 1
start 2
epoch: 2 loss: 1.46904 accuracy: 0.992055
start 3
start 4
epoch: 4 loss: 1.46876 accuracy: 0.992618
start 5
start 6
epoch: 6 loss: 1.4694 accuracy: 0.991891
start 7
start 8
epoch: 8 loss: 1.46841 accuracy: 0.992945
start 9
start 10
epoch: 10 loss: 1.46807 accuracy: 0.9932
start 11
start 12
epoch: 12 loss: 1.46798 accuracy: 0.993382
start 13
start 14
epoch: 14 loss: 1.4682 accuracy: 0.993091
start 15
start 16
epoch: 16 loss: 1.46726 accuracy: 0.993909
start 17
start 18
epoch: 18 loss: 1.46903 accuracy: 0.9922
start 19
start 20
epoch: 20 loss: 1.46748 accuracy: 0.993782
start 21
start 22
epoch: 22 loss: 1.46692 accuracy: 0.994436
start 23
start 24
epoch: 24 loss: 1.46739 accuracy: 0.993982
start 25
start 26
epoch: 26 loss: 1.46756 accuracy: 0.993818
start 27
start 28
epoch: 28 loss: 1.46629 accuracy: 0.995
start 29
start 30
epoch: 30 loss: 1.46647 accuracy: 0.994873
Test accuracy: 0.9907


In [11]:
for epoch in range(31):
    print "start "+str(epoch)
    for batch in range(batches):
        sess.run(train_step, feed_dict={x: x_train[batch_size*batch:batch_size*(batch+1)], y: y_train[batch_size*batch:batch_size*(batch+1)], keep_prob: 0.6})
    if epoch%2==0:
        print "epoch: "+str(epoch)+" loss: "+str(sess.run(loss, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))+" accuracy: "+str(sess.run(accuracy, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))
print "Test accuracy: "+str(sess.run(accuracy, feed_dict={x: x_test, y: y_test, keep_prob: 1.0}))

start 0
epoch: 0 loss: 1.46653 accuracy: 0.994764
start 1
start 2
epoch: 2 loss: 1.46636 accuracy: 0.994909
start 3
start 4
epoch: 4 loss: 1.46597 accuracy: 0.995236
start 5
start 6
epoch: 6 loss: 1.46605 accuracy: 0.995291
start 7
start 8
epoch: 8 loss: 1.46606 accuracy: 0.9952
start 9
start 10
epoch: 10 loss: 1.46582 accuracy: 0.995382
start 11
start 12
epoch: 12 loss: 1.46601 accuracy: 0.995236
start 13
start 14
epoch: 14 loss: 1.46559 accuracy: 0.995636
start 15
start 16
epoch: 16 loss: 1.46576 accuracy: 0.995455
start 17
start 18
epoch: 18 loss: 1.46624 accuracy: 0.995018
start 19
start 20
epoch: 20 loss: 1.46569 accuracy: 0.995582
start 21
start 22
epoch: 22 loss: 1.4653 accuracy: 0.995945
start 23
start 24
epoch: 24 loss: 1.46507 accuracy: 0.996236
start 25
start 26
epoch: 26 loss: 1.46522 accuracy: 0.995982
start 27
start 28
epoch: 28 loss: 1.46491 accuracy: 0.996273
start 29
start 30
epoch: 30 loss: 1.465 accuracy: 0.996236
Test accuracy: 0.9914


In [12]:
for epoch in range(31):
    print "start "+str(epoch)
    for batch in range(batches):
        sess.run(train_step, feed_dict={x: x_train[batch_size*batch:batch_size*(batch+1)], y: y_train[batch_size*batch:batch_size*(batch+1)], keep_prob: 0.6})
    if epoch%2==0:
        print "epoch: "+str(epoch)+" loss: "+str(sess.run(loss, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))+" accuracy: "+str(sess.run(accuracy, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))
print "Test accuracy: "+str(sess.run(accuracy, feed_dict={x: x_test, y: y_test, keep_prob: 1.0}))

start 0
epoch: 0 loss: 1.46502 accuracy: 0.996145
start 1
start 2
epoch: 2 loss: 1.46467 accuracy: 0.996582
start 3
start 4
epoch: 4 loss: 1.4648 accuracy: 0.996418
start 5
start 6
epoch: 6 loss: 1.46464 accuracy: 0.996564
start 7
start 8
epoch: 8 loss: 1.46467 accuracy: 0.996509
start 9
start 10
epoch: 10 loss: 1.46434 accuracy: 0.996873
start 11
start 12
epoch: 12 loss: 1.46444 accuracy: 0.996745
start 13
start 14
epoch: 14 loss: 1.46438 accuracy: 0.9968
start 15
start 16
epoch: 16 loss: 1.46473 accuracy: 0.9966
start 17
start 18
epoch: 18 loss: 1.46456 accuracy: 0.996564
start 19
start 20
epoch: 20 loss: 1.46427 accuracy: 0.996964
start 21
start 22
epoch: 22 loss: 1.46407 accuracy: 0.997091
start 23
start 24
epoch: 24 loss: 1.4641 accuracy: 0.997055
start 25
start 26
epoch: 26 loss: 1.46396 accuracy: 0.9972
start 27
start 28
epoch: 28 loss: 1.46409 accuracy: 0.997055
start 29
start 30
epoch: 30 loss: 1.46397 accuracy: 0.997218
Test accuracy: 0.9921


In [13]:
for epoch in range(31):
    print "start "+str(epoch)
    for batch in range(batches):
        sess.run(train_step, feed_dict={x: x_train[batch_size*batch:batch_size*(batch+1)], y: y_train[batch_size*batch:batch_size*(batch+1)], keep_prob: 0.6})
    if epoch%2==0:
        print "epoch: "+str(epoch)+" loss: "+str(sess.run(loss, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))+" accuracy: "+str(sess.run(accuracy, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))
print "Test accuracy: "+str(sess.run(accuracy, feed_dict={x: x_test, y: y_test, keep_prob: 1.0}))

start 0
epoch: 0 loss: 1.46388 accuracy: 0.997291
start 1
start 2
epoch: 2 loss: 1.464 accuracy: 0.9972
start 3
start 4
epoch: 4 loss: 1.46404 accuracy: 0.997145
start 5
start 6
epoch: 6 loss: 1.4637 accuracy: 0.997509
start 7
start 8
epoch: 8 loss: 1.46379 accuracy: 0.997345
start 9
start 10
epoch: 10 loss: 1.46379 accuracy: 0.997364
start 11
start 12
epoch: 12 loss: 1.46377 accuracy: 0.997455
start 13
start 14
epoch: 14 loss: 1.46382 accuracy: 0.997364
start 15
start 16
epoch: 16 loss: 1.46372 accuracy: 0.997491
start 17
start 18
epoch: 18 loss: 1.46365 accuracy: 0.997564
start 19
start 20
epoch: 20 loss: 1.46359 accuracy: 0.997545
start 21
start 22
epoch: 22 loss: 1.46361 accuracy: 0.997564
start 23
start 24
epoch: 24 loss: 1.46349 accuracy: 0.997691
start 25
start 26
epoch: 26 loss: 1.46346 accuracy: 0.997709
start 27
start 28
epoch: 28 loss: 1.46368 accuracy: 0.997527
start 29
start 30
epoch: 30 loss: 1.46348 accuracy: 0.997673
Test accuracy: 0.9922


In [14]:
for epoch in range(31):
    print "start "+str(epoch)
    for batch in range(batches):
        sess.run(train_step, feed_dict={x: x_train[batch_size*batch:batch_size*(batch+1)], y: y_train[batch_size*batch:batch_size*(batch+1)], keep_prob: 0.6})
    if epoch%2==0:
        print "epoch: "+str(epoch)+" loss: "+str(sess.run(loss, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))+" accuracy: "+str(sess.run(accuracy, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))
print "Test accuracy: "+str(sess.run(accuracy, feed_dict={x: x_test, y: y_test, keep_prob: 1.0}))

start 0
epoch: 0 loss: 1.4635 accuracy: 0.997673
start 1
start 2
epoch: 2 loss: 1.46342 accuracy: 0.997764
start 3
start 4
epoch: 4 loss: 1.46344 accuracy: 0.997727
start 5
start 6
epoch: 6 loss: 1.46332 accuracy: 0.997855
start 7
start 8
epoch: 8 loss: 1.46358 accuracy: 0.997655
start 9
start 10
epoch: 10 loss: 1.46328 accuracy: 0.997909
start 11
start 12
epoch: 12 loss: 1.46328 accuracy: 0.997873
start 13
start 14
epoch: 14 loss: 1.46334 accuracy: 0.997836
start 15
start 16
epoch: 16 loss: 1.46343 accuracy: 0.997727
start 17
start 18
epoch: 18 loss: 1.46338 accuracy: 0.997782
start 19
start 20
epoch: 20 loss: 1.46339 accuracy: 0.997782
start 21
start 22
epoch: 22 loss: 1.46323 accuracy: 0.997927
start 23
start 24
epoch: 24 loss: 1.46326 accuracy: 0.997891
start 25
start 26
epoch: 26 loss: 1.46323 accuracy: 0.997964
start 27
start 28
epoch: 28 loss: 1.46325 accuracy: 0.997891
start 29
start 30
epoch: 30 loss: 1.46321 accuracy: 0.997964
Test accuracy: 0.9927


In [15]:
for epoch in range(31):
    print "start "+str(epoch)
    for batch in range(batches):
        sess.run(train_step, feed_dict={x: x_train[batch_size*batch:batch_size*(batch+1)], y: y_train[batch_size*batch:batch_size*(batch+1)], keep_prob: 0.6})
    if epoch%5==0:
        print "epoch: "+str(epoch)+" loss: "+str(sess.run(loss, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))+" accuracy: "+str(sess.run(accuracy, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))
print "Test accuracy: "+str(sess.run(accuracy, feed_dict={x: x_test, y: y_test, keep_prob: 1.0}))

start 0
epoch: 0 loss: 1.46319 accuracy: 0.997982
start 1
start 2
start 3
start 4
start 5
epoch: 5 loss: 1.46313 accuracy: 0.998055
start 6
start 7
start 8
start 9
start 10
epoch: 10 loss: 1.46305 accuracy: 0.998109
start 11
start 12
start 13
start 14
start 15
epoch: 15 loss: 1.46312 accuracy: 0.998073
start 16
start 17
start 18
start 19
start 20
epoch: 20 loss: 1.46304 accuracy: 0.998145
start 21
start 22
start 23
start 24
start 25
epoch: 25 loss: 1.46301 accuracy: 0.998182
start 26
start 27
start 28
start 29
start 30
epoch: 30 loss: 1.463 accuracy: 0.998182
Test accuracy: 0.9937


In [16]:
for epoch in range(31):
    print "start "+str(epoch)
    for batch in range(batches):
        sess.run(train_step, feed_dict={x: x_train[batch_size*batch:batch_size*(batch+1)], y: y_train[batch_size*batch:batch_size*(batch+1)], keep_prob: 0.6})
    if epoch%5==0:
        print "epoch: "+str(epoch)+" loss: "+str(sess.run(loss, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))+" accuracy: "+str(sess.run(accuracy, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))
print "Test accuracy: "+str(sess.run(accuracy, feed_dict={x: x_test, y: y_test, keep_prob: 1.0}))

start 0
epoch: 0 loss: 1.46298 accuracy: 0.998182
start 1
start 2
start 3
start 4
start 5
epoch: 5 loss: 1.46299 accuracy: 0.998182
start 6
start 7
start 8
start 9
start 10
epoch: 10 loss: 1.46293 accuracy: 0.998236
start 11
start 12
start 13
start 14
start 15
epoch: 15 loss: 1.46298 accuracy: 0.998164
start 16
start 17
start 18
start 19
start 20
epoch: 20 loss: 1.46291 accuracy: 0.998255
start 21
start 22
start 23
start 24
start 25
epoch: 25 loss: 1.46287 accuracy: 0.998327
start 26
start 27
start 28
start 29
start 30
epoch: 30 loss: 1.46287 accuracy: 0.998273
Test accuracy: 0.9938


In [17]:
for epoch in range(31):
    print "start "+str(epoch)
    for batch in range(batches):
        sess.run(train_step, feed_dict={x: x_train[batch_size*batch:batch_size*(batch+1)], y: y_train[batch_size*batch:batch_size*(batch+1)], keep_prob: 0.6})
    if epoch%10==0:
        print "epoch: "+str(epoch)+" loss: "+str(sess.run(loss, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))+" accuracy: "+str(sess.run(accuracy, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))
print "Test accuracy: "+str(sess.run(accuracy, feed_dict={x: x_test, y: y_test, keep_prob: 1.0}))

start 0
epoch: 0 loss: 1.46288 accuracy: 0.998273
start 1
start 2
start 3
start 4
start 5
start 6
start 7
start 8
start 9
start 10
epoch: 10 loss: 1.46282 accuracy: 0.998327
start 11
start 12
start 13
start 14
start 15
start 16
start 17
start 18
start 19
start 20
epoch: 20 loss: 1.46275 accuracy: 0.9984
start 21
start 22
start 23
start 24
start 25
start 26
start 27
start 28
start 29
start 30
epoch: 30 loss: 1.46274 accuracy: 0.9984
Test accuracy: 0.9932


In [18]:
for epoch in range(31):
    print "start "+str(epoch)
    for batch in range(batches):
        sess.run(train_step, feed_dict={x: x_train[batch_size*batch:batch_size*(batch+1)], y: y_train[batch_size*batch:batch_size*(batch+1)], keep_prob: 0.6})
    if epoch%10==0:
        print "epoch: "+str(epoch)+" loss: "+str(sess.run(loss, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))+" accuracy: "+str(sess.run(accuracy, feed_dict={x: x_train, y: y_train, keep_prob: 1.0}))
print "Test accuracy: "+str(sess.run(accuracy, feed_dict={x: x_test, y: y_test, keep_prob: 1.0}))

start 0
epoch: 0 loss: 1.46274 accuracy: 0.998418
start 1
start 2
start 3
start 4
start 5
start 6
start 7
start 8
start 9
start 10
epoch: 10 loss: 1.46267 accuracy: 0.998527
start 11
start 12
start 13
start 14
start 15
start 16
start 17
start 18
start 19
start 20
epoch: 20 loss: 1.46271 accuracy: 0.998455
start 21
start 22
start 23
start 24
start 25
start 26
start 27
start 28
start 29
start 30
epoch: 30 loss: 1.46263 accuracy: 0.998527
Test accuracy: 0.9927
